<a href="https://colab.research.google.com/github/alexandrebhte/Projeto_Imers-o_Aula04/blob/main/Imersao_AI_Aura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [3]:
# Import the Python SDK
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Exemplos de embeddding

In [5]:
title="A próxima geração de IA para desenvolvedores e Google Workapace"
sample_text=("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
"/n"
"Artigo completo: \n"
"/n"
"Gemini APi & Google Ai Studio: uma maneira de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.037058704, -0.026823701, -0.017440202, 0.01763648, 0.07161832, -0.015389419, -0.022126187, -0.025168642, 0.053530004, 0.05680208, 0.014269527, 0.020918477, -0.049980626, -0.019988446, -0.0026882028, -0.016381593, 0.01588024, -0.02116272, -0.03713595, -0.0019989838, 0.0063204914, 0.008964017, -0.020524241, -0.06897993, -0.030079661, 0.011997317, 0.021458264, -0.033330508, -0.034002397, 0.012368606, -0.04998702, 0.056024, -0.043191586, 0.0079679955, -0.041701995, -0.038654868, -0.03368086, -0.047994066, 0.007219825, 0.0013814617, -0.0004085512, -0.0812243, -0.016601713, 0.02869179, -0.0021671155, -0.022736311, 0.040259853, 0.038455263, 0.020596193, -0.062422372, 0.035613075, 0.020018019, 0.064844295, -0.03905805, -0.009430649, -0.0022224546, 0.024021897, -0.039919876, 0.020593455, 5.0364568e-05, 0.007142495, 0.0200164, -0.013243725, 0.05263472, 0.015944842, -0.06740332, -0.034506213, 0.003370163, 0.016599454, 0.04580143, 0.0025630146, 0.00788559, 0.05268026, -0.061465356

Listagem de documentos que serão buscados

In [20]:
DOCUMENT1 = {
    "Titulo":"Operação do sistema de controle climático",
    "Conteúdo":"O Googlecar tem uma sistema de controle climático que permite ajustar a termperatura e o fluco de ar no carro. Para operar o sistemas"
}

DOCUMENT2 = {
    "Titulo":"Touchscreen",
    "Conteúdo":"O seu Googlecar tem uma grande tela sensível ao toque que forneced acesso a uma variedade de recursos, incluindo navegação,  "}

DOCUMENT3 = {
    "Titulo":"Mudança de marchas",
    "Conteúdo":"Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posiçaõ desejada"}

documents = [DOCUMENT1,DOCUMENT2,DOCUMENT3]


In [14]:
df = pd.DataFrame(documents)
df.columns = ["Titulo","Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem uma sistema de controle climát...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [15]:
model = "models/embedding-001"

Criar nossas funçoes

In [16]:
def embed_fn(title, text):
  return  genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]


Criar nova coluna, lambda para rodar linha por linha

In [17]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem uma sistema de controle climát...,"[-0.017852163, -0.017377127, -0.045238964, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.01226155, -0.050604787, -0.02186461, -0.014..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.000695447, -0.037690155, 0.00021905496, 0...."


Criar um função para gerar um embedding da pergunta

In [18]:
def gerar_e_buscar_consulta(consulta,base,model):
 embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]
 produtos_escalares = np.dot(np.stack(df["Embeddings"]),embedding_da_consulta)

 indice = np.argmax(produtos_escalares)

 return df.iloc[indice]["Conteudo"]

Vamos fazer uma consulta

In [22]:
consulta = "Como faço para trocar a marchas em um carro do google?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmcio para a posiçaõ desejada


In [30]:
generation_config ={
		"candidate_count": 1,
		"temperature": 0.5
	}


In [31]:
prompt= f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não faça parte do texto: {trecho}"
model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Bora lá, seu Googlecar tem um câmbio maneiro! É só dar um toque na alavanca e pronto, ele troca as marchas pra você, de boa!
